<img src="logo.png" width="200" align="center">

<center> <h1>Ridge Regression (L2 Regularization)</h1> </center>


## Ridge Regression Solution
In the Ridge Regression (L2 Regularization) [video](https://youtu.be/skOcLw_fXDs) we derived the solution to be:

\begin{equation}
\hat{\theta} = (X^TX)^{-1}X^TXY
\end{equation}

where
\begin{equation}
X = [\bar{x}^T_1, \bar{x}^T_2, ... , \bar{x}^T_n]^T
\end{equation}

\begin{equation}
Y = [y_1, y_2, ... , y_n]^T
\end{equation}

This solution minimizes the following cost function

\begin{equation}
J(x, \theta, y) = \sum_{i=1}^{m}(\theta^T\bar{x}_i - y_i)^2 + \lambda ||\theta||^2
\end{equation}


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [ ]:
class Ridge:
    """Linear least squares with L2 regularization."""
    
    def __init__(self, lam):
        """Initialize a Ridge object.
        
        Args:
            lam: the regularization factor 
        """
        

    def _x_bar(self, x):
        """Create the vector x_bar.
        
        Args:
            x: input vector
        """
    
    def fit(self, x_train, y_train):
        """Generate a fit for the data.
        
        Args:
            x_train: the input values of the training data
            y_train: the output values of the training data
        """


In [ ]:
"""Generate fake data"""
c2 = 0.01
c1 = 1.3
c0 = 3.456
input_var = np.linspace(-10.0, 20.2, 200.0)
output_var = c1 * input_var ** 2 + c1 * input_var + c0 + 500.0 * np.random.rand(len(input_var))

%matplotlib notebook
plt.figure()
plt.scatter(input_var, output_var)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
"""Train using the custom Ridge class"""
x_train, x_test, y_train, y_test = train_test_split(input_var, output_var, test_size=0.20)

lam = 0.1
ridge = Ridge(lam)
ridge.fit(x_train, y_train)

custom_params = ridge._coeff_hat

In [ ]:
"""Train using Sklearn ridge model"""

from sklearn import linear_model

reg = linear_model.Ridge(alpha=lam)
reg.fit([np.array([1.0, x, x**2]) for x in x_train], y_train)

In [ ]:
"""Plot test data and model predictions"""

plt.figure()
plt.scatter(x_test, y_test)
x_test_sorted = np.sort(x_test)
plt.plot(x_test_sorted,
         ridge._coeff_hat[0] + ridge._coeff_hat[1]*x_test_sorted + ridge._coeff_hat[2]*x_test_sorted**2,
         '-r', label='custom')
plt.plot(x_test_sorted,
         reg.intercept_ + reg.coef_[1]*x_test_sorted + reg.coef_[2]*x_test_sorted**2, '--g', label='sklearn')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

# print the coeff
print(f'custom: {ridge._coeff_hat[0]}, {ridge._coeff_hat[1]}, {ridge._coeff_hat[2]}')
print(f'sklearn: {reg.intercept_ }, {reg.coef_[1]}, {reg.coef_[2]}')

In [ ]:
"""Effect of regularization factor"""

coeff_store = []
norm_store = []
factors = np.linspace(0.0, 1.0, 50)
for l in factors:
    ridge = Ridge(l)
    ridge.fit(x_train, y_train)

    coeff_store.append(ridge._coeff_hat)
    norm_store.append(np.linalg.norm(ridge._coeff_hat))

In [ ]:
plt.figure()
plt.subplot(411)
coeff_0 = [c[0] for c in coeff_store]
plt.plot(factors, coeff_0, '-r')
plt.ylabel('c0')
plt.subplot(412)
coeff_1 = [c[1] for c in coeff_store]
plt.plot(factors, coeff_1, 'g')
plt.ylabel('c1')
plt.subplot(413)
coeff_2 = [c[2] for c in coeff_store]
plt.plot(factors, coeff_2, 'b')
plt.ylabel('c3')
plt.subplot(414)
plt.plot(factors, norm_store)
plt.xlabel('lambda')
plt.ylabel('Norm')
plt.show()